In [2]:
%pwd

'c:\\MEDS\\EDS-213\\bren-eds213-data'

In [3]:
%ls

 Volume in drive C is OS
 Volume Serial Number is 6C74-3388

 Directory of c:\MEDS\EDS-213\bren-eds213-data

05/06/2025  10:00 AM    <DIR>          .
04/16/2025  10:34 PM    <DIR>          ..
04/09/2025  11:21 PM                44 .gitignore
04/10/2025  09:37 AM                 0 .Rhistory
04/09/2025  11:21 PM    <DIR>          .Rproj.user
04/22/2025  09:55 AM    <DIR>          .vscode
04/24/2025  09:40 AM    <DIR>          ASDN_csv
04/09/2025  11:21 PM            87,274 asdn-er-diagram.png
04/10/2025  09:36 AM               267 bren-eds213-data.Rproj
04/29/2025  12:26 PM    <DIR>          database
04/24/2025  09:48 AM            12,288 database.db
04/29/2025  12:27 PM            28,678 discussion-datacleaning-JORGENSEN.ipynb
04/17/2025  08:45 PM    <DIR>          discussion-section
04/29/2025  12:27 PM               274 export_data.sql
04/29/2025  12:27 PM             1,703 import_snow_cover.sql
04/09/2025  11:53 PM             2,234 JORGENSEN-data-modeling.qmd
04/23/2025  09:51 PM   

In [4]:
cd database/

c:\MEDS\EDS-213\bren-eds213-data\database


In [5]:
%pip install duckdb

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.4 MB 330.3 kB/s eta 0:00:35
   ---------------------------------------- 0.1/11.4 MB 525.1 kB/s eta 0:00:22
   - -------------------------------------- 0.4/11.4 MB 3.1 MB/s eta 0:00:04
   --- ------------------------------------ 1.1/11.4 MB 7.0 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.4 MB 8.9 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.4 MB 11.2 MB/s eta 0:00:01
   ---------- ----------------------------- 3.0/11.4 MB 10.0 MB/s eta 0:00:01
   -------------- ------------------------- 4.0/11.4 MB 11.7 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.4 MB 12.6 MB/s eta 0:00:01
   ------------------- -------------------- 5.7/11.4 MB 13.4 MB/s eta 0:00:01
   --------------------- ------------------ 6.2/11.4 MB 13.7 MB/s eta 0:0

In [2]:
import duckdb

In [3]:
conn = duckdb.connect('database.db')

In [4]:
cur = conn.cursor()

In [5]:
cur.execute("SELECT * FROM Site LIMIT 3")

CatalogException: Catalog Error: Table with name Site does not exist!
Did you mean "sqlite_master"?

LINE 1: SELECT * FROM Site LIMIT 3
                      ^

In [10]:
# Method 1: getting all at once using fetchall()
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375)]

In [11]:
# Cursors dont save through results, they are just pass thru mechanisms
# Cursros always return a list of tubles, even 1 tuples
cur.execute("SELECT Code FROM Site LIMIT 3")
cur.fetchall()

[('barr',), ('burn',), ('bylo',)]

In [12]:
cur.execute('SELECT Code FROM Site LIMIT 3')

In [13]:
[t[0] for t in cur.fetchall()]

['barr', 'burn', 'bylo']

In [14]:
# Method 2: getting one at a time using fetchone()
cur.execute("SELECT Code FROM Site")
cur.fetchone()

('barr',)

In [15]:
cur.fetchone()

('burn',)

In [16]:
cur.fetchone()

('bylo',)

In [ ]:
cur.execute('SELECT Code FROM Site')
for row in cur:
    print(row[0]) # this wont work

TypeError: 'duckdb.duckdb.DuckDBPyConnection' object is not iterable

In [18]:
cur.execute('''CREATE TEMP TABLE t as
            SELECT * FROM Bird_nests WHERE Nest_ID LIKE '14%' ''')

In [19]:
cur.fetchall()

[(99,)]

In [21]:
cur.execute('SELECT COUNT(*) FROM t')
cur.fetchone()[0]

99

In [22]:
# A better practice: spell it out, dont do a select 
cur.execute('SELECT Code, Latitude, Longitude FROM Site Limit 3')
cur.fetchall()

[('barr', 71.30000305175781, -156.60000610351562),
 ('burn', 55.20000076293945, -84.30000305175781),
 ('bylo', 73.19999694824219, -80.0)]

In [23]:
# parameterized queries
cur.execute('SELECT Code FROM Species LIMIT 3 ')
cur.fetchall()

[('agsq',), ('amcr',), ('amgp',)]

In [ ]:
# Hypothesize: we want to know the # of nests for each species and no group by
species = 'agsq'
query = 'SELECT COUNT(*) FROM Bird_nests WHERE Species = ?'
cur.execute(query, [species])
cur.fetchall() # there are 0 for this species

[(0,)]

In [25]:
# Lets put two things together: query for all species, and loop over these species, getting th number of nuests for each species
cur.execute('SELECT Code FROM Species')
for row in cur.fetchall(): # in any other database, could iterate over the cursor
    species_code = row[0]
    cur2 = conn.cursor()
    cur2.execute(query, [species_code])
    num_nests = cur2.fetchone()[0]
    print(f"Species {species_code} has {num_nests} nests")
    cur2.close() # close the cursor to free up resources

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests
Species arfo has 0 nests
Species arte has 0 nests
Species basa has 0 nests
Species bbis has 0 nests
Species bbpl has 43 nests
Species bbsa has 0 nests
Species besw has 0 nests
Species bltu has 0 nests
Species brant has 0 nests
Species brbe has 0 nests
Species brle has 0 nests
Species btcu has 0 nests
Species btgo has 3 nests
Species cole has 0 nests
Species cora has 0 nests
Species cosn has 0 nests
Species crpl has 2 nests
Species cusa has 0 nests
Species dunl has 101 nests
Species eywa has 0 nests
Species glgu has 0 nests
Species goea has 0 nests
Species gwfg has 0 nests
Species gwgu has 0 nests
Species gwte has 0 nests
Species gyrf has 0 nests
Species herg has 3 nests
Species hore has 0 nests
Species hugo has 0 nests
Species kill has 0 nests
Species lalo has 33 nests
Species lbdo has 1 nests
Species lesa has 0 nests
Species leye has 0 nests
Species list has 0 nests
Species ltdu has 0 nests
Species ltja has 0 

In [26]:
import pandas as pd

In [27]:
df = pd.read_sql('SELECT * FROM Site', conn)

C:\Users\baile\AppData\Local\Temp\ipykernel_25612\2685156110.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM Site', conn)


In [28]:
df

,Code,Site_name,Location,Latitude,Longitude,Area
0,barr,Barrow,"Alaska, USA",71.300003,-156.600006,220.399994
1,burn,Burntpoint Creek,"Ontario, Canada",55.200001,-84.300003,63.000000
2,bylo,Bylot Island,"Nunavut, Canada",73.199997,-80.000000,723.599976
3,cakr,Cape Krusenstern,"Alaska, USA",67.099998,-163.500000,54.099998
4,cari,Canning River Delta,"Alaska, USA",70.099998,-145.800003,722.000000
5,chau,Chaun River Delta,"Chukotka, Russia",68.800003,170.600006,248.199997
6,chur,Churchill,"Manitoba, Canada",58.700001,-93.800003,866.900024
7,coat,Coats Island,"Nunavut, Canada",62.900002,-82.500000,1239.099976
8,colv,Colville River Delta,"Alaska, USA",70.400002,-150.699997,324.799988
9,eaba,East Bay,"Nunavut, Canada",64.000000,-81.699997,1205.500000


In [6]:
conn.close()